# GenAI with Python: Agents from scratch

###### [Article: TowardsDataScience]()

### 0 - Setup

In [ ]:
#pip install langchain --> 0.2.14
#pip install langgraph --> 0.2.19

##### LLM

In [64]:
from langchain_community.llms import Ollama #0.2.12

llm = Ollama(model="llama3.1")

res = llm.invoke(input=['''What's the paper number of Attention is all you need?''']).split("\n")[0]
print(res)

The paper "Attention Is All You Need" by Vaswani et al. has a publication title but I am unable to verify its paper number or where it was published.


### 1 - Tools

### 3 - Single Agent

In [ ]:
import typing
from langchain_core.messages import BaseMessage
from langchain_core.agents import AgentAction
import operator

class AgentState(typing.TypedDict):
    input: str #user's most recent query
    chat_history: list[BaseMessage] #full chat
    intermediate_steps: typing.Annotated[list[tuple[AgentAction, str]], operator.add] #record of all actions it does 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = """
You are the oracle, the great AI decision maker.
Given the user's query you must decide what to do with it based on the list of tools provided to you.

If you see that a tool has been used (in the scratchpad) with a particular query, do NOT use that same tool with the same query again. 
Also, do NOT use any tool more than twice (ie, if the tool appears in the scratchpad twice, do not use it again).

You should aim to collect information from a diverse range of sources before providing the answer to the user. 
Once you have collected plenty of information to answer the user's question (stored in the scratchpad) use the final_answer tool.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("assistant", "scratchpad: {scratchpad}")
])

### 4 - Multiple Agents

In [36]:
from langchain_community.agent_toolkits.load_tools import load_tools, get_all_tool_names

print( get_all_tool_names() )

tools = load_tools(llm=llm, tool_names=[
                    "wikipedia", #1.4.0
                    "ddg-search", #6.2.11
                    "requests_get",
                   ], allow_dangerous_tools=True)

['sleep', 'wolfram-alpha', 'google-search', 'google-search-results-json', 'searx-search-results-json', 'bing-search', 'metaphor-search', 'ddg-search', 'google-lens', 'google-serper', 'google-scholar', 'google-finance', 'google-trends', 'google-jobs', 'google-serper-results-json', 'searchapi', 'searchapi-results-json', 'serpapi', 'dalle-image-generator', 'twilio', 'searx-search', 'merriam-webster', 'wikipedia', 'arxiv', 'golden-query', 'pubmed', 'human', 'awslambda', 'stackexchange', 'sceneXplain', 'graphql', 'openweathermap-api', 'dataforseo-api-search', 'dataforseo-api-search-json', 'eleven_labs_text2speech', 'google_cloud_texttospeech', 'read_file', 'reddit_search', 'news-api', 'tmdb-api', 'podcast-api', 'memorize', 'llm-math', 'open-meteo-api', 'requests', 'requests_get', 'requests_post', 'requests_patch', 'requests_put', 'requests_delete', 'terminal']


In [37]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools=tools, llm=llm, verbose=True)

res = agent.invoke("What day is today and how's the weather in Milan?", 
                   handle_parsing_errors=True)



> Entering new AgentExecutor chain...


ValueError: 'llama3.1' did not respond with valid JSON. 
                Please try again. 
                Response: { "tool": "requests_get", "tool_input": { "url": "https://www.google.com/search?q=weather+Milan" }

In [47]:
from langchain_core.tools import tool
import requests
import re

@tool("tool_arxiv")
def tool_arxiv(arxiv_id: str) -> str:
    """Gets the abstract from an ArXiv paper given the ID. Useful for finding high-level context about a specific paper."""
    res = requests.get(f"https://export.arxiv.org/abs/{arxiv_id}")
    # search html for abstract
    get_abstract = re.compile(
        r'<blockquote class="abstract mathjax">\s*<span class="descriptor">Abstract:</span>\s*(.*?)\s*</blockquote>',
        re.DOTALL)
    res = get_abstract.search(res.text).group(1)
    return res

print( tool_arxiv("1706.03762") )

The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, based
solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to be
superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014
English-to-German translation task, improving over the existing best results,
including ensembles by over 2 BLEU. On the WMT 2014 English-to-French
translation task, our model establishes a new single-model state-of-the-art
BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction
of the training costs of the best models from the literature. We show that the
Transforme

In [62]:
import wikipedia

@tool("tool_wikipedia")
def tool_wikipedia(q: str) -> str:
    """Search on Wikipedia"""
    return wikipedia.summary(q)

print( tool_wikipedia("Attention is all you need") )

"Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, based on the attention mechanism proposed in 2014 by Bahdanau et al. It is considered a foundational paper in modern artificial intelligence, as the transformer approach has become the main architecture of large language models like those based on GPT. At the time, the focus of the research was on improving Seq2seq techniques for machine translation, but the authors go further in the paper, foreseeing the technique's potential for other tasks like question answering and what is now known as multimodal Generative AI.
The paper's title is a reference to the song "All You Need Is Love" by the Beatles. The name "Transformer" was picked because Uszkoreit liked the sound of that word.
An early design document was titled "Transformers: Iterative Self-Attention and Processing for Various

In [54]:
from langchain_community.tools import DuckDuckGoSearchRun

@tool("tool_browser")
def tool_browser(q: str) -> str:
    """Search on DuckDuckGo browser"""
    return DuckDuckGoSearchRun().run(q)

@tool("tool_instagram")
def tool_instagram(q: str) -> str:
    '''Search on Instagram'''
    return DuckDuckGoSearchRun().run(f"site:instagram.com {q}")

print('\x1b[1;31m'+'Browser:'+'\x1b[0m', tool_browser("Attention is all you need") )
print('\n\x1b[1;31m'+''+'Instagram:'+'\x1b[0m', tool_instagram("Attention is all you need") )

Browser: 8 Google Employees Invented Modern AI. Here's the Inside ... The Transformer. Attention Is All You Need Figure 1. The goal is to understand why the Transformer was so groundbreaking and how it achieves these capabilities by implementing each part manually, relying only on NumPy without any out-of-the-box packages from Keras or PyTorch. Being a deep-learning beginner myself, I will try to add as much ... The transformer model, introduced with the groundbreaking paper 'Attention is All You Need', has revolutionized NLP by shifting the paradigm from sequential processing to parallel attention ... The transformer is an architecture that relies on the concept of attention, a technique used to provide weights to different parts of an input sequence so that a better understanding of its ... The "Attention is All You Need" paper ushered in an era where AI could focus, prioritize, and synthesize information in a manner more akin to human cognition, leading to advanced generative ...

I

In [ ]:
@tool("final_answer")
def final_answer(introduction: str, steps: str, main_body: str, conclusion: str, sources: str):
    """Returns a natural language response to the user in the form of a report. 
    There are several sections to this report, those are:
    - `introduction`: a short paragraph introducing the user's question and the topic we are researching.
    - `steps`: a few bullet points explaining the steps that were taken to research your report.
    - `main_body`: this is where the bulk of high-quality and concise information that answers the user's question belongs. 
    It is 3-4 paragraphs long in length.
    - `conclusion`: this is a short single paragraph conclusion providing a concise but sophisticated view on what was found.
    - `sources`: a bulletpoint list providing detailed sources for all information referenced during the research process
    """
    if type(steps) is list:
        steps = "\n".join([f"- {r}" for r in steps])
    if type(sources) is list:
        sources = "\n".join([f"- {s}" for s in sources])
    return ""